In [1]:
!gsutil cp gs://cs327e-open-access/load_testing/us-500.zip .
!gsutil cp gs://cs327e-open-access/load_testing/us-1000000.zip .

Copying gs://cs327e-open-access/load_testing/us-500.zip...
/ [1 files][ 37.1 KiB/ 37.1 KiB]                                                
Operation completed over 1 objects/37.1 KiB.                                     
Copying gs://cs327e-open-access/load_testing/us-1000000.zip...
- [1 files][ 69.7 MiB/ 69.7 MiB]                                                
Operation completed over 1 objects/69.7 MiB.                                     


In [2]:
!unzip us-500.zip
!unzip us-1000000.zip

Archive:  us-500.zip
  inflating: us-500.csv              
Archive:  us-1000000.zip
  inflating: us-1000000.csv          


In [21]:
!pip install mysql-connector-python

In [1]:
import pandas as pd
import csv

In [2]:
%env INSTANCE = mysql-final-project

env: INSTANCE=mysql-final-project


In [3]:
!mysql < /home/jupyter/create_table.sql

In [4]:
us_500_results = pd.read_csv("us-500.csv")
us_500_results.head()

,first_name,last_name,company_name,address,city,county,state,zip,phone1,phone2,email,web
0,James,Butt,"Benton, John B Jr",6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com
1,Josephine,Darakjy,"Chanay, Jeffrey A Esq",4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,http://www.chanayjeffreyaesq.com
2,Art,Venere,"Chemel, James L Cpa",8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,8014,856-636-8749,856-264-4130,art@venere.org,http://www.chemeljameslcpa.com
3,Lenna,Paprocki,Feltz Printing Service,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010,lpaprocki@hotmail.com,http://www.feltzprintingservice.com
4,Donette,Foller,Printing Dimensions,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561,donette.foller@cox.net,http://www.printingdimensions.com


In [5]:
us_1000000_results = pd.read_csv("us-1000000.csv")
us_1000000_results.head()

,first_name,last_name,company_name,address,city,county,state,zip,phone1,phone2,email,web
0,James,Butt,"Benton, John B Jr",6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com
1,Josephine,Darakjy,"Chanay, Jeffrey A Esq",4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,http://www.chanayjeffreyaesq.com
2,Art,Venere,"Chemel, James L Cpa",8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,8014,856-636-8749,856-264-4130,art@venere.org,http://www.chemeljameslcpa.com
3,Lenna,Paprocki,Feltz Printing Service,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010,lpaprocki@hotmail.com,http://www.feltzprintingservice.com
4,Donette,Foller,Printing Dimensions,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561,donette.foller@cox.net,http://www.printingdimensions.com


In [6]:
print(len(us_1000000_results))

1000000


In [7]:
print("Column Name" + " --> " + "Max Length of Characters")
for column in us_500_results:
    print(column, " --> ", us_500_results[column].astype(str).str.len().max())

Column Name --> Max Length of Characters
first_name  -->  10
last_name  -->  13
company_name  -->  30
address  -->  31
city  -->  19
county  -->  20
state  -->  2
zip  -->  5
phone1  -->  12
phone2  -->  12
email  -->  34
web  -->  42


In [22]:
%%timeit
import mysql.connector

connection = mysql.connector.connect(
  host="10.128.0.5",
  user="root",
  password="codesnroses",
  database="load_testing",
  autocommit=False
)

print(connection)

sql = "INSERT INTO Person (first_name, last_name, company_name, address, city, county, state, zip, phone1, phone2, email, web)\
	  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
#rec = ("James","Butt","Benton, John B Jr","6649 N Blue Gum St",\
#	  "New Orleans","Orleans","LA","70116","504-621-8927","504-845-1427","jbutt@gmail.com","http://www.bentonjohnbjr.com")
cursor = connection.cursor()
numRecords = 0
filename = "us-1000000.csv"
with open(filename, "r") as csvfile:
    datareader = csv.reader(csvfile)
    next(datareader) #skip first row of column headings 
    for row in datareader:
        
        try:
            cursor.execute(sql, row)
            numRecords += 1
            #print(numRecords)
            if(numRecords % 5000 == 0):
                connection.commit()
                print(numRecords, " Number Record inserted successfully into Person table")

        except mysql.connector.Error as error:
            print("Failed to insert record into Person table {}".format(error))

        finally:
            if connection.is_connected() and numRecords == 1000000:
                connection.close()
                print("MySQL connection is closed")
                !mysql load_testing -e "select count(*) from Person"
                

5000  Number Record inserted successfully into Person table
10000  Number Record inserted successfully into Person table
15000  Number Record inserted successfully into Person table
20000  Number Record inserted successfully into Person table
25000  Number Record inserted successfully into Person table
30000  Number Record inserted successfully into Person table
35000  Number Record inserted successfully into Person table
40000  Number Record inserted successfully into Person table
45000  Number Record inserted successfully into Person table
50000  Number Record inserted successfully into Person table
55000  Number Record inserted successfully into Person table
60000  Number Record inserted successfully into Person table
65000  Number Record inserted successfully into Person table
70000  Number Record inserted successfully into Person table
75000  Number Record inserted successfully into Person table
80000  Number Record inserted successfully into Person table
85000  Number Record inse

In [23]:
!mysql load_testing -e "select count(*) from Person"

+----------+
| count(*) |
+----------+
|  8020000 |
+----------+
